In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

In [5]:

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from imblearn.metrics import specificity_score # specificity_score 임포트
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Flatten
import seaborn as sns
from scipy.spatial.distance import euclidean
import random
import numpy as np

In [6]:
# 1. 데이터 로드
df_X_train = pd.read_csv('/content/drive/My Drive/PhalangesOutlinesCorrect/X_train_sample2.csv')
df_y_train = pd.read_csv('/content/drive/My Drive/PhalangesOutlinesCorrect/y_train_sample2.csv')
df_X_test = pd.read_csv('/content/drive/MyDrive/PhalangesOutlinesCorrect/X_test_original.csv')
df_y_test = pd.read_csv('/content/drive/MyDrive/PhalangesOutlinesCorrect/y_test_original.csv')


X_train = df_X_train.values
y_train = df_y_train.values.reshape(-1)  # 1차원 배열(벡터)로 변환
X_test = df_X_test.values
y_test = df_y_test.values.reshape(-1)  # 1차원 배열(벡터)로 변환


In [7]:
# 소수 클래스와 다수 클래스 식별
target_counts = pd.Series(y_train).value_counts()
minority_class = target_counts.idxmin()
majority_class = target_counts.idxmax()

# 소수 클래스 샘플 추출
X_minority = X_train[y_train == minority_class]
X_major = X_train[y_train == majority_class]

In [8]:
from dtaidistance import dtw
from scipy.spatial.distance import euclidean
import random
from tqdm import tqdm
from dtaidistance.dtw import distance  # 함수 이름은 distance임

In [9]:
from dtaidistance.dtw import distance as dtw_distance  # 함수 이름을 바꿔서 import

def calculate_all_dtw_distances(X):
    distances = []
    n = len(X)
    for i in range(n):
        for j in range(i + 1, n):
            dist_val = dtw_distance(X[i], X[j])  # 함수 호출
            distances.append((i, j, dist_val))   # 결과 저장
    return distances


In [10]:
# DTW 거리 계산 실행
X_all = np.vstack((X_minority, X_major))  # 전체 학습 데이터
y_all = np.hstack((np.full(len(X_minority), minority_class), np.full(len(X_major), majority_class)))

print(f"전체 샘플 수: {len(X_all)}, 소수 클래스: {minority_class}, 다수 클래스: {majority_class}")

# 모든 샘플 간의 DTW 거리 계산
dtw_distances = calculate_all_dtw_distances(X_all)

# 거리 행렬 초기화
n_samples = len(X_all)
dtw_matrix = np.zeros((n_samples, n_samples))

# 거리 행렬 채우기
for i, j, dist_val in dtw_distances:
    dtw_matrix[i, j] = dist_val
    dtw_matrix[j, i] = dist_val  # 대칭 행렬

print("DTW 거리 행렬 계산 완료:", dtw_matrix.shape)


전체 샘플 수: 246, 소수 클래스: 0, 다수 클래스: 1
DTW 거리 행렬 계산 완료: (246, 246)


In [11]:
k = 3  # 홀수로 설정 (예: 3, 5, 7 등)

safe_minority_indices = []  # 안전 샘플 인덱스
risky_minority_indices = []  # 위험 샘플 인덱스

minority_indices = np.where(y_all == minority_class)[0]
majority_indices = np.where(y_all == majority_class)[0]

for idx in minority_indices:
    distances = dtw_matrix[idx]

    # 자기 자신 제외하고 가까운 이웃 k개 선택
    nearest_indices = np.argsort(distances)[1:k+1]

    # 이웃의 클래스 확인
    neighbor_labels = y_all[nearest_indices]
    minority_count = np.sum(neighbor_labels == minority_class)
    majority_count = np.sum(neighbor_labels == majority_class)

    # 안전 / 위험 분류
    if minority_count < majority_count:
        safe_minority_indices.append(idx)
    else:
        risky_minority_indices.append(idx)

print(f"총 소수 클래스 수: {len(minority_indices)}")
print(f"안전 샘플 수: {len(safe_minority_indices)}")
print(f"위험 샘플 수: {len(risky_minority_indices)}")


총 소수 클래스 수: 12
안전 샘플 수: 11
위험 샘플 수: 1


In [12]:
# 6. 안전 샘플에 해당하는 소수 클래스 샘플만을 가지고 증강 준비
X_safe_minority = X_all[safe_minority_indices]

# 7. 증강할 데이터 수 계산
n_major = len(X_major)
n_minority = len(X_minority)
n_safe = len(X_safe_minority)
n_augment = n_major - n_minority

print(f"증강해야 할 샘플 수: {n_augment}, 안전 소수 클래스 샘플 수: {n_safe}")

증강해야 할 샘플 수: 222, 안전 소수 클래스 샘플 수: 11


In [13]:

# 8. 한 쌍에서 생성할 수와 나머지 계산
n_per_pair = n_augment // n_safe
remainder = n_augment % n_safe
print(f"각 안전 샘플당 생성 수: {n_per_pair}, 남는 개수: {remainder}")


각 안전 샘플당 생성 수: 20, 남는 개수: 2


In [14]:
augmented_samples = []

for idx, safe_idx in enumerate(safe_minority_indices):
    safe_sample = X_all[safe_idx]

    # DTW 거리 기반 이웃 찾기
    distances = dtw_matrix[safe_idx]
    nearest_indices = np.argsort(distances)[1:k+1]  # 자기 자신 제외, k개의 이웃

    # 이웃 중 다수 클래스에 해당하는 인덱스만 추출
    majority_neighbors = [i for i in nearest_indices if y_all[i] == majority_class]

    if not majority_neighbors:
        continue  # 다수 클래스 이웃이 없다면 증강 생략

    for _ in range(n_per_pair + (1 if idx < remainder else 0)):
        # 이웃 중 다수 클래스 샘플 무작위 선택
        majority_neighbor_idx = random.choice(majority_neighbors)
        majority_sample = X_all[majority_neighbor_idx]

        # 랜덤 선형 보간
        alpha = np.random.rand()
        synthetic_sample = alpha * safe_sample + (1 - alpha) * majority_sample
        augmented_samples.append(synthetic_sample)


In [15]:
# 12. 최종 증강 데이터와 레이블 결합
X_aug = np.array(augmented_samples)
y_aug = np.full(len(X_aug), minority_class)

print(f"증강된 샘플 수: {len(X_aug)}")

# 13. 기존 학습 데이터와 결합
X_train_augmented = np.vstack((X_train, X_aug))
y_train_augmented = np.hstack((y_train, y_aug))

print("증강된 학습 데이터 최종 크기:", X_train_augmented.shape, y_train_augmented.shape)

증강된 샘플 수: 222
증강된 학습 데이터 최종 크기: (468, 80) (468,)


In [16]:
def predict_with_logistic_regression(X_train, y_train, X_test):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_cart(X_train, y_train, X_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_knn(X_train, y_train, X_test, k=3):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_xgboost(X_train, y_train, X_test):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    return model.predict(X_test)

def predict_with_lstm(X_train, y_train, X_test):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0)
    return (model.predict(X_test_reshaped) > 0.5).astype(int)

def predict_with_cnn(X_train, y_train, X_test):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0)
    return (model.predict(X_test_reshaped) > 0.5).astype(int)


In [17]:
# 예측 실행 및 평가
predictions = {
    "Logistic Regression": predict_with_logistic_regression(X_train_augmented, y_train_augmented, X_test),
    "CART": predict_with_cart(X_train_augmented, y_train_augmented, X_test),
    "KNN": predict_with_knn(X_train_augmented, y_train_augmented, X_test),
    "XGBoost": predict_with_xgboost(X_train_augmented, y_train_augmented, X_test),
    "LSTM": predict_with_lstm(X_train_augmented, y_train_augmented, X_test),
    "CNN": predict_with_cnn(X_train_augmented, y_train_augmented, X_test)
}
# 성능 지표 저장을 위한 리스트
results = []

for model_name, pred_y in predictions.items():
    accuracy = accuracy_score(y_test, pred_y)
    recall = recall_score(y_test, pred_y)
    f1 = f1_score(y_test, pred_y)
    specificity = specificity_score(y_test, pred_y)
    conf_matrix = confusion_matrix(y_test, pred_y)
    results.append([accuracy, f1, recall, specificity, conf_matrix])


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:27:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(*

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [18]:
# 성능 지표를 DataFrame으로 변환
results_df = pd.DataFrame(results, columns=["Accuracy", "F1", "Recall", "Specificity", "Confusion Matrix"], index=predictions.keys())
model_results = results_df.T
# 결과를 출력
print("\n모델 성능 비교 결과:")
print(model_results)

# 결과를 CSV 파일로 저장
model_results.to_csv("/content/drive/My Drive/PhalangesOutlinesCorrect/results/bsmote_soda_result.csv")


모델 성능 비교 결과:
                     Logistic Regression                     CART  \
Accuracy                        0.610723                 0.652681   
F1                              0.723051                 0.753719   
Recall                          0.828897                  0.86692   
Specificity                      0.26506                 0.313253   
Confusion Matrix  [[88, 244], [90, 436]]  [[104, 228], [70, 456]]   

                                       KNN                 XGBoost  \
Accuracy                          0.670163                0.652681   
F1                                0.749336                0.766458   
Recall                            0.804183                0.929658   
Specificity                       0.457831                0.213855   
Confusion Matrix  [[152, 180], [103, 423]]  [[71, 261], [37, 489]]   

                                     LSTM                       CNN  
Accuracy                         0.581585                  0.610723  
F1        